In [1]:
!pip install openpyxl
!pip install mne
import pandas as pd
import mne
import json

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [31]:
file_errors_location = 'EinsatzdatenBeatmung.xlsx'
df = pd.read_excel(file_errors_location)
df.head()

,# T,Index,DateTime,Pmean,Ppeak,Pplat,etCO2,f,MVe,Vte,...,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110
0,# S,Index,DateTime,peep,pInsp,pInspCcsv,vt,freq,patientType,ventilationMode,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,# A,Index,DateTime,Pinput_low,Pinput_high,peep_high,Paw_low,apnoe,flow_unrealisable,temp_high,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,I,2,13.12.2021 14:49:27,activeView=1,airmix=no,allowMaintenanceSuppress=1,allowMaintSuppress=1,apnoe=30,audioVolume=50,autoAlarmLimit=30,...,triggerInsp=5,triggerOn=0,triggerType=simple,triggerWindow=20,uptime=22714,ventilationMode=IPPV,ventPauseTime=50,version=1,vt=1300,vtKg=6
3,S,3,13.12.2021 14:49:27,5,20,60,1300,10,adult,IPPV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,4,13.12.2021 14:49:27,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
ventilation_file = "2021-12-13_14-49-27_MMS2_SN14638.edf"
ventilation_data = mne.io.read_raw_edf(ventilation_file)
ventilation_raw_data = data.get_data()
# you can get the metadata included in the file and a list of all channels:
ventilation_info = data.info
ventilation_channels = data.ch_names

ekg_file = "recording.edf"
ekg_data = mne.io.read_raw_edf(ekg_file)
ekg_raw_data = data.get_data()
# you can get the metadata included in the file and a list of all channels:
ekg_info = data.info
ekg_channels = data.ch_names

Extracting EDF parameters from /home/jupyter/2021-12-13_14-49-27_MMS2_SN14638.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jupyter/recording.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [11]:
ch1, ch2, ch3, ch4, ch5, ch6, ch7, ch8, ch9 = [], [], [], [], [], [], [], [], []
for i in ventilation_raw_data[0]:
    ch1.append(i)
    
for i in ventilation_raw_data[1]:
    ch2.append(i)
    
for i in ventilation_raw_data[2]:
    ch3.append(i)

for i in ventilation_raw_data[3]:
    ch4.append(i)
    
for i in ekg_raw_data[0]:
    ch5.append(i)

for i in ekg_raw_data[1]:
    ch6.append(i)

for i in ekg_raw_data[2]:
    ch7.append(i)

for i in ekg_raw_data[3]:
    ch8.append(i)
    
for i in ekg_raw_data[4]:
    ch9.append(i)

In [10]:
ekg_channels

['s0', 's1', 's2', 's3', 'HF']

In [12]:
dictionarylist = [
    {
        "name": "PAW",
        "data": ch1,
        "resolution": 100
    },
    {
        "name": "EXTFLOWBTPS",
        "data": ch2,
        "resolution": 100
    },
    
    {
        "name": "FLOWBTPS",
        "data": ch3,
        "resolution": 100
    },
    {
        "name": "CO2_WAVE",
        "data": ch4, 
        "resolution": 40
    },
    {
        "name": "EKG_CHANNEL_2",
        "data": ch5, 
        "resolution": 500
    },
    {
        "name": "EKG_CHANNEL_3",
        "data": ch6, 
        "resolution": 500
    },
    {
        "name": "NULL",
        "data": ch7, 
        "resolution": 500
    },
    {
        "name": "PLETH",
        "data": ch8, 
        "resolution": 500
    },
    {
        "name": "HF",
        "data": ch9, 
        "resolution": 1
    },
]

In [13]:
with open('data.json', 'w') as f:
    json.dump({"data": dictionarylist}, f)

[9.765923032318825e-07,
 9.765923032318825e-07,
 -9.765923032318825e-07,
 -0.0001904354991302242,
 -0.00012207403790398885,
 -3.222754600665212e-05,
 6.543168431653613e-05,
 0.00013183996093631123,
 0.00017871639149144157,
 0.00012988677632984748,
 4.394665364543471e-05,
 -3.808709982604342e-05,
 -9.668263801995991e-05,
 -0.00014551225318155402,
 -0.0001084017456587425,
 -2.0508438367869532e-05,
 7.519760734885494e-05,
 0.000116214484084594,
 0.00027051606799523856,
 0.0006103701895199407,
 0.0008799096652119544,
 0.0007763908810693678,
 0.00045216223639637577,
 0.00020996734519486182,
 0.0001552781762138764,
 -9.765923032318825e-07,
 -0.0003759880367442854,
 -0.0006884975737784984,
 -0.000670918912320321,
 -0.0004970854823450424,
 -0.0003916135135959955,
 -0.000354503006073184,
 -0.0003056733909115863,
 -0.00025684377574999216,
 -0.00013965269936216273,
 6.836146122623177e-06,
 0.00014355906857509382,
 0.00023731192968535453,
 0.00023145237586596324,
 0.0001279335917233837,
 -4.394665